In [1]:
import pandas as pd
import numpy as np
import os

from itertools import product

In [2]:
def trata_dados(df, ano, pop_name):
    df['ID_Municipio'] = df['Município'].str[:6] #coluna do codigo do municipio
    df['Município'] = df['Município'].str[7:] #coluna do nome do municipio
    df['ano'] = ano #coluna do ano dos dados

    df.columns = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total', 'ID_Municipio', 'ano']
        
    df['ID_Municipio'] = pd.to_numeric(df['ID_Municipio'])
        
    #nao sei explicar direito o que faz, mas deixa tudo no formato desejado
    df = df.melt(id_vars=['Município', 'ID_Municipio', 'ano'], var_name='faixa_etaria', value_name=pop_name)
    
    return df

In [3]:
#gera o df com a populacao total padrao de 2010

cols = ['Município', '0a4', '5a9', '10a14', '15a19', '20a24', '25a29',
              '30a34', '35a39', '40a44', '45a49', '50a54', '55a59', '60a64', 
              '65a69', '70a74', '75a79', '>=80anos', 'Total']

df = pd.read_csv("dados populacionais/dados total/2010total.csv", encoding='latin1', sep=';')

padrao_t = df.iloc[[-1]]
padrao_t.columns = cols

padrao_t = padrao_t.melt(id_vars=['Município'], var_name='faixa_etaria', value_name="pop_pad_total")
padrao_t.drop(labels=['Município'], axis=1, inplace=True)

#gera o df com a populacao masculina padrao de 2010

df = pd.read_csv("dados populacionais/dados homem/homem2010.csv", encoding='latin1', sep=';')

padrao_h = df.iloc[[-1]]
padrao_h.columns = cols

padrao_h = padrao_h.melt(id_vars=['Município'], var_name='faixa_etaria', value_name="pop_pad_homem")
padrao_h.drop(labels=['Município'], axis=1, inplace=True)

#gera o df com a populacao feminina padrao de 2010

df = pd.read_csv("dados populacionais/dados mulher/mulher2010.csv", encoding='latin1', sep=';')

padrao_m = df.iloc[[-1]]
padrao_m.columns = cols

padrao_m = padrao_m.melt(id_vars=['Município'], var_name='faixa_etaria', value_name="pop_pad_mulher")
padrao_m.drop(labels=['Município'], axis=1, inplace=True)

In [4]:
padrao_t

,faixa_etaria,pop_pad_total
0,0a4,1338553
1,5a9,1497792
2,10a14,1711110
3,15a19,1742648
4,20a24,1757058
5,25a29,1727867
6,30a34,1617371
7,35a39,1435712
8,40a44,1392455
9,45a49,1312183


In [5]:
#populacao total
df = pd.read_csv("dados populacionais/dados total/2010total.csv", encoding='latin1', sep=';')
df.drop(df.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

df = trata_dados(df, 2010, "pop_total")

#populacao masculina
df_h = pd.read_csv("dados populacionais/dados homem/homem2010.csv", encoding='latin1', sep=';')
df_h.drop(df_h.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

df_h = trata_dados(df_h, 2010, "pop_homem")

#populacao feminina
df_m = pd.read_csv("dados populacionais/dados mulher/mulher2010.csv", encoding='latin1', sep=';')
df_m.drop(df_m.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

df_m = trata_dados(df_m, 2010, "pop_mulher")

In [6]:
directory = os.fsencode("./dados populacionais/dados total/") #populacao total

anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

i = 0

#cria o dataframe com populacao total
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename != '2010total.csv':
        aux = pd.read_csv("./dados populacionais/dados total/" + filename, encoding='latin1', sep=";")
        aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

        aux = trata_dados(aux, anos[i], "pop_total")

        df = df.append(aux)
        
        i = i + 1 #avanca o index de anos

In [7]:
directory = os.fsencode("./dados populacionais/dados homem/") #populacao masculina

anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

i = 0

#cria o dataframe com populacao masculina
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename != 'homem2010.csv':
        aux = pd.read_csv("./dados populacionais/dados homem/" + filename, encoding='latin1', sep=";")
        aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

        aux = trata_dados(aux, anos[i], "pop_homem")

        df_h = df_h.append(aux)
        
        i = i + 1 #avanca o index de anos

In [8]:
directory = os.fsencode("./dados populacionais/dados mulher/") #populacao feminina

anos = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

i = 0

#cria o dataframe com populacao feminina
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename != 'mulher2010.csv':
        aux = pd.read_csv("./dados populacionais/dados mulher/" + filename, encoding='latin1', sep=";")
        aux.drop(aux.tail(1).index, inplace=True) #remove a linha com os dados agregados do estado (indesejada)

        aux = trata_dados(aux, anos[i], "pop_mulher")

        df_m = df_m.append(aux)
        
        i = i + 1 #avanca o index de anos

In [9]:
df = df.merge(padrao_t, how="inner")

df = df.merge(df_h, how="inner")
df = df.merge(padrao_h, how="inner")

df = df.merge(df_m, how="inner")
df = df.merge(padrao_m, how="inner")

df

,Município,ID_Municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher
0,Abadia dos Dourados,310010,2010,0a4,418,1338553,211,684629,207,653924
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924
2,Abre Campo,310030,2010,0a4,914,1338553,464,684629,450,653924
3,Acaiaca,310040,2010,0a4,279,1338553,135,684629,144,653924
4,Açucena,310050,2010,0a4,687,1338553,368,684629,319,653924
...,...,...,...,...,...,...,...,...,...,...
153535,Virginópolis,317180,2019,Total,10510,19957444,5180,9828254,5330,10129190
153536,Virgolândia,317190,2019,Total,5380,19957444,2622,9828254,2758,10129190
153537,Visconde do Rio Branco,317200,2019,Total,42564,19957444,21194,9828254,21370,10129190
153538,Volta Grande,317210,2019,Total,5252,19957444,2512,9828254,2740,10129190


In [10]:
#usado para obter os nomes de todos os agravos e seus subgrupos
agrav = pd.read_csv("Planilha para o R - Planilha.csv")
agrav = agrav.drop(labels='Unnamed: 0', axis=1)
agrav = agrav.fillna(0)
agrav.columns = ['cidsize', 'cid', 'agravo', 'subgrupo']

#listas com o nomes das doenças, anos, e meses
nomes_agravo = np.concatenate((agrav['agravo'].unique(), agrav['subgrupo'].unique()[1:]), axis=None)
faixas = df['faixa_etaria'].unique()
faixas = faixas[:-1]
anos = [ano for ano in range(2010, 2020)]
meses = [mes for mes in range(1, 13)]

In [11]:
#grupos de cidades analisadas
cidades_selecionadas = ["Betim", 'Brumadinho','Curvelo','Esmeraldas','Florestal','Fortuna de Minas',
                        'Igarapé','Juatuba','Maravilhas','Mário Campos','Martinho Campos',
                        'Papagaios','Pará de Minas','Paraopeba','Pequi','Pompéu','São Joaquim de Bicas',
                        'São José da Varginha','Sarzedo']

cidades_limitrofes = ['Abaeté','Araçaí','Belo Horizonte','Belo Vale','Bom Despacho','Bonfim','Cachoeira da Prata',
                      'Caetanópolis','Capim Branco','Conceição do Pará','Contagem','Cordisburgo','Corinto',
                      'Dores do Indaiá','Felixlândia','Ibirité','Igaratinga','Inhaúma','Inimutaba','Itaúna',
                      'Itabirito','Itatiaiuçu','Leandro Ferreira','Mateus Leme','Matozinhos','Moeda',
                      'Morada Nova de Minas','Morro da Garça','Nova Lima','Onça de Pitangui','Paineiras',
                      'Pedro Leopoldo','Pitangui','Presidente Juscelino','Quartel Geral','Ribeirão das Neves',
                      'Rio Manso','Santana de Pirapama','Santo Hipólito','Sete Lagoas']


cidades_prox_limitrofes = ['Araújos','Augusto de Lima','Baldim','Biquinhas','Carmo do Cajuru','Cedro do Abaeté',
                           'Conceição do Mato Dentro','Confins','Congonhas','Congonhas do Norte','Crucilândia',
                           'Estrela do Indaiá','Funilândia','Gouveia','Itaguara','Jaboticatubas','Jeceaba',
                           'Jequitibá','Lagoa Santa','Lassance','Luz','Moema','Monjolos','Nova Serrana',
                           'Ouro Preto','Piedade dos Gerais','Prudente de Morais',
                           'Raposos','Rio Acima','Sabará','São Gonçalo do Abaeté','São Gonçalo do Pará',
                           'São Gotardo','São José da Lapa','Santa Bárbara','Santa Luzia','Santana do Riacho',
                           'Santo Antônio do Monte','Serra da Saudade','Tiros','Vespasiano']


#gera uma lista com o ID de todas as cidades a serem avaliadas
todas = cidades_selecionadas + cidades_limitrofes + cidades_prox_limitrofes

cidades = df[df['Município'].isin(todas)]['ID_Municipio'].unique()

In [12]:
#cria um dataframe vazio, que armazenará todas as permutações de municipio, ano, mês e agravo
df_size = len(list(product(cidades, faixas, anos, meses, nomes_agravo)))
df_index = [i for i in range(df_size)]

agregados = pd.DataFrame({'id_municipio': pd.Series([], dtype='float32'),
                            'faixa_etaria': pd.Series([], dtype='str'),
                            'ano': pd.Series([], dtype='float32'),
                            'mes': pd.Series([], dtype='float32'),
                            'agravo': pd.Series([], dtype='str')},
                            index = df_index)

agregados

,id_municipio,faixa_etaria,ano,mes,agravo
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
12851995,NaN,NaN,NaN,NaN,NaN
12851996,NaN,NaN,NaN,NaN,NaN
12851997,NaN,NaN,NaN,NaN,NaN
12851998,NaN,NaN,NaN,NaN,NaN


In [13]:
k = 0

data = agregados.values

for row in product(cidades, faixas, anos, meses, nomes_agravo):
    data[k] = row

    k = k + 1

In [14]:
np.savetxt("agregados.csv", data, fmt='%s', header="ID_Municipio faixa_etaria ano mes agravo", comments='')

In [15]:
agregados = pd.read_csv("agregados.csv", sep=" ")
agregados

,ID_Municipio,faixa_etaria,ano,mes,agravo
0,310020,0a4,2010,1,colera
1,310020,0a4,2010,1,diarreia
2,310020,0a4,2010,1,odip
3,310020,0a4,2010,1,tubresp
4,310020,0a4,2010,1,tbsistnerv
...,...,...,...,...,...
12851995,317120,>=80anos,2019,12,bronquite
12851996,317120,>=80anos,2019,12,asma
12851997,317120,>=80anos,2019,12,broncoeq
12851998,317120,>=80anos,2019,12,infecpele


In [16]:
sim = pd.read_csv("sim_agravos.csv", encoding="latin1")

In [17]:
#obitos de agravos por faixa etaria municipio ano e mes
aux = sim[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
aux.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_total']

#adiciona os agravos considerados como "subgrupos" na tabela
aux_sub = sim[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
aux_sub.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_total']

aux = aux.append(aux_sub, ignore_index=True)
aux

,ID_Municipio,faixa_etaria,ano,mes,agravo,obitos_total
0,310000,0a4,2010,11,infecespperi,1
1,310000,15a19,2016,2,veneno,1
2,310000,15a19,2016,5,veneno,1
3,310000,15a19,2016,10,veneno,1
4,310000,15a19,2018,4,veneno,1
...,...,...,...,...,...,...
432616,317220,>=80anos,2016,6,hipertensao,1
432617,317220,>=80anos,2017,2,bronquite,2
432618,317220,>=80anos,2017,3,bronquite,1
432619,317220,>=80anos,2018,2,iamrecor,1


In [18]:
#obitos por homem
sim_h = sim[sim['sexo2'] == 'M']

aux_h = sim_h[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
aux_h.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_homem']

aux_sub_h = sim_h[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
aux_sub_h.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_homem']

aux_h = aux_h.append(aux_sub_h, ignore_index=True)

#obitos por mulher
sim_m = sim[sim['sexo2'] == 'F']

aux_m = sim_m[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'agravo'], as_index=False).count()
aux_m.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_mulher']

aux_sub_m = sim_m[['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo', 'IDADE']].groupby(['CODMUNRES', 'faixaetaria', 'ano', 'mes', 'subgrupo'], as_index=False).count()
aux_sub_m.columns = ['ID_Municipio', 'faixa_etaria', 'ano', 'mes', 'agravo', 'obitos_mulher']

aux_m = aux_m.append(aux_sub_m, ignore_index=True)

In [19]:
aux = aux.merge(aux_h, how='left')
aux = aux.merge(aux_m, how='left')

In [20]:
taxas = agregados.merge(aux, how='left')

taxas = taxas.fillna(0)
taxas = taxas.astype({'obitos_total': 'int32',
                        'obitos_homem': 'int32',
                        'obitos_mulher': 'int32'})

taxas

,ID_Municipio,faixa_etaria,ano,mes,agravo,obitos_total,obitos_homem,obitos_mulher
0,310020,0a4,2010,1,colera,0,0,0
1,310020,0a4,2010,1,diarreia,0,0,0
2,310020,0a4,2010,1,odip,0,0,0
3,310020,0a4,2010,1,tubresp,0,0,0
4,310020,0a4,2010,1,tbsistnerv,0,0,0
...,...,...,...,...,...,...,...,...
12851995,317120,>=80anos,2019,12,bronquite,1,0,1
12851996,317120,>=80anos,2019,12,asma,0,0,0
12851997,317120,>=80anos,2019,12,broncoeq,0,0,0
12851998,317120,>=80anos,2019,12,infecpele,0,0,0


In [21]:
taxas = df.merge(taxas, how='inner')

taxas['taxa_obitos_total'] = (taxas['obitos_total'] / taxas['pop_total']) * 100000
taxas['taxa_obitos_homem'] = (taxas['obitos_homem'] / taxas['pop_homem']) * 100000
taxas['taxa_obitos_mulher'] = (taxas['obitos_mulher'] / taxas['pop_mulher']) * 100000

taxas['obitos_esp_total'] = taxas['pop_pad_total'] * (taxas['taxa_obitos_total'] / 100000)
taxas['obitos_esp_homem'] = taxas['pop_pad_homem'] * (taxas['taxa_obitos_homem'] / 100000)
taxas['obitos_esp_mulher'] = taxas['pop_pad_mulher'] * (taxas['taxa_obitos_mulher'] / 100000)

taxas['taxa_obitos_esp_total'] = (taxas['obitos_esp_total'] / taxas['pop_pad_total']) * 100000
taxas['taxa_obitos_esp_homem'] = (taxas['obitos_esp_homem'] / taxas['pop_pad_homem']) * 100000
taxas['taxa_obitos_esp_mulher'] = (taxas['obitos_esp_mulher'] / taxas['pop_pad_mulher']) * 100000

In [22]:
taxas['grupo'] = np.nan

taxas.loc[(taxas['Município'].isin(cidades_selecionadas)), 'grupo'] = 1
taxas.loc[(taxas['Município'].isin(cidades_limitrofes)), 'grupo'] = 2
taxas.loc[(taxas['Município'].isin(cidades_prox_limitrofes)), 'grupo'] = 3

taxas = taxas[~taxas['grupo'].isnull()]

In [23]:
'''cod_faixas = {
    '0a4': 1,
    '5a9': 2,
    '10a14': 3, 
    '15a19': 4,
    '20a24': 5,
    '25a29': 6,
    '30a34': 7,
    '35a39': 8,
    '40a44': 9,
    '45a49': 10,
    '50a54': 11,
    '55a59': 12,
    '60a64': 13,
    '65a69': 14,
    '70a74': 15,
    '75a79': 16,
    '>=80anos': 17
}

taxas['cod_faixas'] = np.nan
taxas["cod_faixas"] = taxas["faixa etaria"].apply(lambda x: cod_faixas.get(x))

taxas '''

'cod_faixas = {\n    \'0a4\': 1,\n    \'5a9\': 2,\n    \'10a14\': 3, \n    \'15a19\': 4,\n    \'20a24\': 5,\n    \'25a29\': 6,\n    \'30a34\': 7,\n    \'35a39\': 8,\n    \'40a44\': 9,\n    \'45a49\': 10,\n    \'50a54\': 11,\n    \'55a59\': 12,\n    \'60a64\': 13,\n    \'65a69\': 14,\n    \'70a74\': 15,\n    \'75a79\': 16,\n    \'>=80anos\': 17\n}\n\ntaxas[\'cod_faixas\'] = np.nan\ntaxas["cod_faixas"] = taxas["faixa etaria"].apply(lambda x: cod_faixas.get(x))\n\ntaxas '

In [24]:
taxas.columns = ['municipio', 'id_municipio', 'ano', 'faixa_etaria', 'pop_total',
       'pop_pad_total', 'pop_homem', 'pop_pad_homem', 'pop_mulher',
       'pop_pad_mulher', 'mes', 'agravo', 'obitos_total',
       'obitos_homem', 'obitos_mulher', 'taxa_obitos_total',
       'taxa_obitos_homem', 'taxa_obitos_mulher', 'obitos_esp_total',
       'obitos_esp_homem', 'obitos_esp_mulher', 'taxa_obitos_esp_total',
       'taxa_obitos_esp_homem', 'taxa_obitos_esp_mulher', 'grupo']

taxas

,municipio,id_municipio,ano,faixa_etaria,pop_total,pop_pad_total,pop_homem,pop_pad_homem,pop_mulher,pop_pad_mulher,...,taxa_obitos_total,taxa_obitos_homem,taxa_obitos_mulher,obitos_esp_total,obitos_esp_homem,obitos_esp_mulher,taxa_obitos_esp_total,taxa_obitos_esp_homem,taxa_obitos_esp_mulher,grupo
0,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,2.0
1,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,2.0
2,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,2.0
3,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,2.0
4,Abaeté,310020,2010,0a4,1377,1338553,702,684629,675,653924,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12851995,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,66.357001,0.0,102.145046,221.985401,0.0,209.500511,66.357001,0.0,102.145046,3.0
12851996,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3.0
12851997,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3.0
12851998,Vespasiano,317120,2019,>=80anos,1507,334532,528,129431,979,205101,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3.0


In [25]:
len(taxas.columns)

25

In [26]:
taxas.to_csv('pre_taxa_sim.csv', index=False)